In [1]:
from datasets import load_dataset
import torch
import torch.nn as nn
import transformers
from accelerate import PartialState
from datasets import Dataset, IterableDataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
local_ranks = 3

model_path = "/data/public/model/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right")
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
model.to(local_ranks)

/data/gwy/miniconda3/envs/torch25/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [2]:
from datasets import load_dataset,load_from_disk

train_dataset = load_from_disk("/data4/gwy/R1/understand-r1-zero-main/datasets/train/math_12k/train")

In [3]:
train_dataset[0]

{'problem': 'How many units long is a segment whose endpoints are $(-4,1)$ and $(1,13)$?',
 'solution': 'We use the distance formula: $\\sqrt{(-4 - 1)^2 + (1 - 13)^2},$ which is $\\sqrt{25 + 144} = \\sqrt{169} = \\boxed{13}$.\n\n- OR -\n\nWe note that the points $(-4,1)$, $(1,13)$, and $(1,1)$ form a right triangle with legs of length 5 and 12. $(5,12,13)$ is a Pythagorean triple, so the hypotenuse has length $\\boxed{13}$.',
 'answer': '13',
 'subject': 'Algebra',
 'level': 2,
 'unique_id': 'test/algebra/1570.json',
 'gold_solution_steps': ['We use the distance formula:',
  '$\\sqrt{(-4 - 1)^2 + (1 - 13)^2},$ which is $\\sqrt{25 + 144} = \\sqrt{169} = \\boxed{13}$.',
  '- OR - We note that the points $(-4,1)$, $(1,13)$,',
  'and $(1,1)$ form a right triangle with legs of length 5 and 12. $(5,12,13)$ is a Pythagorean triple, so the hypotenuse has length $\\boxed{13}$.']}

In [4]:
# Format into conversation
def make_r1_conversation(example):
    return {
        "prompt": "A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. "
    "The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.\nUser: "
    + example["problem"]
    + "\nAssistant: <think>",
        "solution": example['solution']
    }

train_dataset = train_dataset.map(make_r1_conversation, remove_columns=train_dataset.column_names, num_proc=8)

In [7]:
train_dataset[0]

{'solution': 'We use the distance formula: $\\sqrt{(-4 - 1)^2 + (1 - 13)^2},$ which is $\\sqrt{25 + 144} = \\sqrt{169} = \\boxed{13}$.\n\n- OR -\n\nWe note that the points $(-4,1)$, $(1,13)$, and $(1,1)$ form a right triangle with legs of length 5 and 12. $(5,12,13)$ is a Pythagorean triple, so the hypotenuse has length $\\boxed{13}$.',
 'prompt': 'A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.\nUser: How many units long is a segment whose endpoints are $(-4,1)$ and $(1,13)$?\nAssistant: <think>'}

In [6]:
from vllm import LLM, SamplingParams

vllm_device = "cuda:3"
llm = LLM(
    model=model_path ,
    device=vllm_device,
    gpu_memory_utilization=0.7,
    dtype="auto",
    enable_prefix_caching=True,
    max_model_len=None,
)
sampling_params = SamplingParams(
temperature=0.9,
max_tokens=1024,
)

2025-03-24 16:52:45,589	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 03-24 16:52:45 __init__.py:207] Automatically detected platform cuda.
INFO 03-24 16:52:50 config.py:549] This model supports multiple tasks: {'score', 'classify', 'embed', 'reward', 'generate'}. Defaulting to 'generate'.
INFO 03-24 16:52:50 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/data/public/model/Qwen2.5-1.5B-Instruct', speculative_config=None, tokenizer='/data/public/model/Qwen2.5-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:3, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.06it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.05it/s]



INFO 03-24 16:52:53 model_runner.py:1115] Loading model weights took 2.8875 GB
INFO 03-24 16:52:54 worker.py:267] Memory profiling takes 0.81 seconds
INFO 03-24 16:52:54 worker.py:267] the current vLLM instance can use total_gpu_memory (79.33GiB) x gpu_memory_utilization (0.70) = 55.53GiB
INFO 03-24 16:52:54 worker.py:267] model weights take 2.89GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 50.52GiB.
INFO 03-24 16:52:54 executor_base.py:111] # cuda blocks: 118254, # CPU blocks: 9362
INFO 03-24 16:52:54 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 57.74x
INFO 03-24 16:52:56 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:11<00:00,  2.93it/s]

INFO 03-24 16:53:08 model_runner.py:1562] Graph capturing finished in 12 secs, took 0.21 GiB
INFO 03-24 16:53:08 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 15.63 seconds


In [8]:
from trl.data_utils import apply_chat_template, is_conversational, maybe_apply_chat_template

device =  "cuda:0"
inputs = train_dataset.select(range(16))
prompts = [x["prompt"] for x in inputs]
prompts_text = [maybe_apply_chat_template(example, tokenizer)["prompt"] for example in inputs]
prompt_inputs = tokenizer(
        prompts_text, return_tensors="pt", padding=True, padding_side="left", add_special_tokens=False
    )
prompt_inputs.to(device)
prompt_ids, prompt_mask = prompt_inputs["input_ids"], prompt_inputs["attention_mask"]
print(prompt_ids.shape, prompt_mask.shape)

torch.Size([16, 228]) torch.Size([16, 228])


In [9]:
prompts_text[0]

'A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.\nUser: How many units long is a segment whose endpoints are $(-4,1)$ and $(1,13)$?\nAssistant: <think>'

In [10]:
outputs = llm.generate(prompts_text, sampling_params=sampling_params, use_tqdm=False)

In [11]:
from trl.trainer.utils import generate_model_card, get_comet_experiment_url, pad, selective_log_softmax

device =  "cuda:0"
completion_ids = [out.token_ids for completions in outputs for out in completions.outputs]

# Pad the completions, and concatenate them with the prompts
completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
completion_ids = pad(completion_ids, padding_value=tokenizer.pad_token_id)
prompt_completion_ids = torch.cat([prompt_ids, completion_ids], dim=1)

In [12]:
print(completion_ids.shape, prompt_completion_ids.shape)

torch.Size([16, 927]) torch.Size([16, 1155])


In [13]:
# Mask everything after the first EOS token
is_eos = completion_ids == tokenizer.eos_token_id
eos_idx = torch.full((is_eos.size(0),), is_eos.size(1), dtype=torch.long, device=device)
eos_idx[is_eos.any(dim=1)] = is_eos.int().argmax(dim=1)[is_eos.any(dim=1)]
sequence_indices = torch.arange(is_eos.size(1), device=device).expand(is_eos.size(0), -1)
completion_mask = (sequence_indices <= eos_idx.unsqueeze(1)).int()


In [14]:
# Concatenate prompt_mask with completion_mask for logit computation
attention_mask = torch.cat([prompt_mask, completion_mask], dim=1)  # (B*G, P+C)
logits_to_keep = completion_ids.size(1)  # we only need to compute the logits for the completion tokens
print(attention_mask.shape, logits_to_keep)

torch.Size([16, 1155]) 927


In [15]:
# with torch.inference_mode():
#     logits = model(input_ids=prompt_completion_ids, attention_mask=attention_mask, logits_to_keep=logits_to_keep + 1).logits
# print(logits.shape)
# logits = logits[:, :-1, :]  # (B, L-1, V), exclude the last logit: it corresponds to the next token pred
# print(logits.shape)
# input_ids = prompt_completion_ids[:, -logits_to_keep:]
# print(input_ids.shape)
# logits = logits[:, -logits_to_keep:]
# print(logits.shape)

In [15]:
# Decode the generated completions
completions_text = tokenizer.batch_decode(completion_ids, skip_special_tokens=True)
prompt_completion_text = tokenizer.batch_decode(prompt_completion_ids,  skip_special_tokens=True)

In [16]:
if is_conversational(inputs[0]):
    completions = []
    for prompt, completion in zip(prompts, completions_text):
        bootstrap = prompt.pop()["content"] if prompt[-1]["role"] == "assistant" else ""
        completions.append([{"role": "assistant", "content": bootstrap + completion}])
else:
    completions = completions_text

In [17]:
from easyalign.reward.openr1_reward import  (
    accuracy_reward,
    format_reward,
    get_cosine_scaled_reward,
    get_repetition_penalty_reward,
    len_reward,
    reasoning_steps_reward,
    tag_count_reward, 
    understand_reward
)
rewards = understand_reward(prompts, completions)

In [28]:
type(rewards[0])

tuple

In [19]:
for content, sol in zip(prompts, completions):
    break

In [22]:
sol

' We have two points, A(-4,1) and B(1,13), and we want to find the length of the segment connecting these two points. To do this, we can use the distance formula: distance = √[(x2-x1)² + (y2-y1)²]  <answer> According to the distance formula, if we substitute A(-4,1) and B(1,13) into the formula, we get distance = √[(1 - (-4))² + (13 - 1)²]  \ndistance = √[5² + 12²]  \ndistance = √[25 + 144]  \ndistance = √169  \ndistance = 13 <answer> </answer> </answer>'

In [21]:
content

'A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.\nUser: How many units long is a segment whose endpoints are $(-4,1)$ and $(1,13)$?\nAssistant: <think>'

In [23]:
if "</think> <answer>" in sol and "</answer>" in sol:
    print(1)

In [18]:
from easyalign.reward.openr1_reward import  (
    accuracy_reward,
    format_reward,
    get_cosine_scaled_reward,
    get_repetition_penalty_reward,
    len_reward,
    reasoning_steps_reward,
)

cosine_min_value_wrong = 0
cosine_max_value_wrong = 0.5
cosine_min_value_correct = -0.5
cosine_max_value_correct = 1.0
cosine_max_len = 1000
repetition_n_grams = 3
repetition_max_penalty = -1
REWARD_FUNCS_REGISTRY = {
    "accuracy": accuracy_reward,
    "format": format_reward,
    "reasoning_steps": reasoning_steps_reward,
    "cosine": get_cosine_scaled_reward(
        min_value_wrong=cosine_min_value_wrong,
        max_value_wrong=cosine_max_value_wrong,
        min_value_correct=cosine_min_value_correct,
        max_value_correct=cosine_max_value_correct,
        max_len=cosine_max_len,
    ),
    "repetition_penalty": get_repetition_penalty_reward(
        ngram_size=repetition_n_grams,
        max_penalty=repetition_max_penalty,
    ),
    "length": len_reward,
}
reward_funcs = [REWARD_FUNCS_REGISTRY[func] for func in ["accuracy", "format"]]

In [19]:
reward_processing_classes = [None] * len(reward_funcs)
rewards_per_func = torch.zeros(len(prompts), len(reward_funcs), device=device)
for i, (reward_func, reward_processing_class) in enumerate(
    zip(reward_funcs, reward_processing_classes)
):
    # Repeat all input columns (but "prompt" and "completion") to match the number of generations
    keys = [key for key in inputs[0] if key not in ["prompt", "completion"]]
    reward_kwargs = {key: [example[key] for example in inputs] for key in keys}
    output_reward_func = reward_func(prompts=prompts, completions=completions, **reward_kwargs)
    rewards_per_func[:, i] = torch.tensor(output_reward_func, dtype=torch.float32, device=device)

 We need for the value of the function at x=-2, x=2, and x=2 to be the same. So, we'll put the value of x=-2 in the first equation, x=2 in the middle equation, and x=2 in the last equation. Let's do that. We get: a(-2)+3 = -2-5 = -7. And a+3 = 0. So, a = -3. The last value we can find out is b, for when x=2, we use the expression from the second equation. So 2(-3)-b = 0, b = -6. Therefore, a+b = -3+(-6) = -9. </think>
<answer>-9</answer>


TypeError: string indices must be integers, not 'str'

In [56]:
rewards_per_func

tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 0.]], device='cuda:0')